# Python "científico": biblioteca _pandas_

![](images/sci_python_pandas.png)

_web site_: (`pandas.pydata.org`)

![](images/pandas_web.png)

## `Series`

> `Series` is a one-dimensional **labeled** array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the **index**. 

In [ ]:
import pandas as pd

Uma Série (_Series_) é um conjunto (ordenado) de valores, mas cada valor é associado a uma "etiqueta" (_label_).

Ao conjunto das etiquetas dá-se o nome de "**índice**".

Quando construímos uma Série, usando a função `Series()`, podemos indicar o índice.

In [ ]:
s = pd.Series([1.4,2.2,3.2,6.5,12], index=['a', 'b', 'c', 'd', 'e'])
print(s)

Se não indicarmos um índice, o conjunto dos inteiros sucessivos será o índice.

In [ ]:
s = pd.Series([1.4,2.2,3.2,6.5,12])
print(s)

As Séries podem ser construídas a partir de um dicionário, em que as chaves são o índice.

In [ ]:
d = {'a' : 0., 'b' : 1., 'c' : 2.}
s = pd.Series(d)
print(s)

Podemos, mesmo neste caso, indicar um índice. Caso o índice tenha elementos para além das chaves do dicionário, haverá **valores em falta**.

In [ ]:
s = pd.Series({'a' : 0., 'b' : 1., 'c' : 2.}, index=['b', 'c', 'd', 'a'])
print(s)

O uso do marcador `NaN` para indicar **valores em falta** e a existência de muitas funções de análise que levam em conta valores em falta são uma característica muito poderosa do módulo `pandas`.

### Indexação e operações vetoriais

As Séries podem ser usadas **como dicionários: as etiquetas comportam-se como chaves** e são usadas para indexar uma Série. para obter um valor (e também para modificar um valor).

Tal como nos dicionários, o operador `in` **testa a existência de uma etiqueta**.

In [ ]:
s = pd.Series(d, index=['b', 'c', 'd', 'a'])
print(s)

print('-----------')
print(s['b'])
print(s.c) # notação abreviada

s['b'] = 0.5

print('z' in s) # teste de existência de um label
print('d' in s)

Mas as Séries são muito mais poderosas: elas comportam-se como _arrays_ do módulo `numpy`. Podemos usar:

- indexação com inteiros
- _slices_
- **operações vetoriais**.

In [ ]:
s = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8}, 
              index=['b', 'c', 'd', 'e', 'a'])
print(s)
print("""-----------
{}

{}

{}

{}
""".format(s[0],
           s[:3],
           s**2,
           s[s > 0.75]))


Também muito poderoso é o facto de que, quando aplicamos operações vetoriais sobre Séries (por exemplo, na soma de duas séries), **os valores são "alinhados" pelos respetivos _labels_** antes da operação. Vejamos estas duas séries:

In [ ]:
s1 = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8})
s2 = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'f': 1.8})

print('  s1')
print(s1)
print('\n  s2')
print(s2)
print('\n  Soma')
print(s1 + s2)

A soma das duas Séries resulta numa Série em que todas as etiquetas estão presentes (**união de conjuntos**).

As que só existirem numa das Séries ou as que, numa das Séries, têm o valor `NaN`, terão o valor `NaN` no resultado final.

A função `.dropna()` permite eliminar os _valores em falta_.

In [ ]:
s1 = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8})
s2 = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'f': 1.8})
s3 = s1 + s2
print('  s3 = s1 + s2')
print(s3)
print('\n  s3.dropna()')
print(s3.dropna())

### Funções descritivas dos valores

As Séries têm algumas funções de estatística descritiva de grande utilidade.

Note-se que, em geral, **os valores em falta são ignorados nos cálculos**.

In [ ]:
s = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8})
print(s)

print('\nMédia:         {}'.format(s.mean()))
print('Desvio padrão: {:5.3f}'.format(s.std()))

In [ ]:
s = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8})
print(s.describe())

In [ ]:
s = pd.Series({'a' : 0.5, 'b' : 1.0, 'c' : 3.0, 'e': 1.8})
print(s.cumsum())

## `DataFrame`

> `DataFrame` is a **2-dimensional labeled data structure** with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a **dict of Series objects**. It is generally the most commonly used pandas object.

Uma _DataFrame_ é um quadro bidimensional, em que cada coluna se comporta como uma Série, mas em que existe um índice comum a todas as colunas.

Para ilustar o uso de uma `DataFrame`, vamos ler e processar a informação da UniProt sobre a levedura _S. cerevisiae_.

A `DataFrame` terá as colunas "**ac**", "**rev**", "**n**" e "**sequence**"

In [ ]:
nome_ficheiro = 'uniprot_proteome_S_cerevisiae.txt'

def get_prots(filename):
    with open(filename) as big:
        tudo = big.read()
    todas = [p for p in tudo.split('//\n') if len(p) != 0]
    return todas

prots = get_prots('uniprot_proteome_S_cerevisiae.txt')

def process_prot(p):
    linhas = p.split('\n')
    linhasid = linhas[0]
    linhaac = linhas[1]
    partes = linhasid.split()
    reviewed = partes[2][0:-1]
    naa = int(partes[3])
    partes = linhaac.split()
    ac = partes[1][0:-1]
    
    for i in range(len(linhas)-1, 0, -1):
        if linhas[i].startswith('SQ'):
            break
    s = ''.join(linhas[i+1:])
    seq = ''.join(s.split())
    return {'ac':ac, 'rev':reviewed, 'n':naa, 'seq':seq}

pinfo = [process_prot(p) for p in prots]
print('Numero total de proteínas: {}'.format(len(pinfo)))
print('\nPrimeira proteína:')
print(pinfo[0])

Podemos construir uma `DataFrame` a partir de uma lista de dicionários. As **chaves dos dicionários serão as colunas**.

In [ ]:
prots = pd.DataFrame(pinfo)
prots

Podemos mudar o índice para uma das colunas.

In [ ]:
prots = prots.set_index('ac')
prots

Para inspeção rápida, as funções `.head()` e `.tail()` apresentam o início e o fim da `DataFrame`

In [ ]:
prots.head()

In [ ]:
prots.tail()

A indexação com o nome de uma coluna devolve essa coluna (mas associada ao índice).

Cada coluna comporta-se como uma Série.

In [ ]:
print(prots['n'])

In [ ]:
print(prots['n']['P31383'])
print(prots['n'].max())
print(prots['n'].min())
print(prots['n'].mean())

In [ ]:
print(prots['n'].describe())

Para obter uma linha usamos `.loc` e indexação por um _label_.

In [ ]:
print(prots.loc['P31383'])

In [ ]:
print(prots.loc['P31383']['seq'].count('W'))

A indexação com condições sobre as colunas é muito poderosa:

In [ ]:
bigs = prots[prots['n'] > 800]
print(len(bigs))
bigs

In [ ]:
bigs = prots[prots['n'] > 800]
bigs['n']

In [ ]:
prots[prots['n'] > 800]['n'].mean()

In [ ]:
prots['n'].idxmax()

In [ ]:
prots.loc[prots['n'].idxmax()]

In [ ]:
prots[prots['n']==prots['n'].max()]

In [ ]:
prots[prots['n']==prots['n'].min()]

Para aplicar funções de _strings_ a toda uma coluna de uma só vez, usamos o atributo `.str.` sobre essa coluna (o resultado é uma Série):

In [ ]:
numW = prots['seq'].str.count('W')
numW

Com uma indexação por nome, podemos inserir uma coluna nova na `DataFrame` (no fim).

In [ ]:
prots['W'] = numW
prots

As `DataFrame`s também têm funções descritivas, mas o facto de cada coluna ser uma Série podemos realizar muitas análises de uma forma simples.

In [ ]:
print(prots.info())

In [ ]:
print(prots['W'].describe())

In [ ]:
print(prots['rev'].value_counts())

In [ ]:
# só no IPython/Jupyter notebook
%matplotlib inline

In [ ]:
import matplotlib.pyplot as pl
p = prots['n'].plot(kind='hist', alpha=0.5, bins=50)
p.set_ylabel('Proteins')
x = p.set_xlabel('Length (aa)')